In [1]:
import numpy as np
import scipy.linalg as LA


## Lab. 5:  Esqueleto

In [8]:
def mDesCyclic( A, b, tol=1e-6,maxiter=1000 ):
    n=len(b)
    assert((n,n) == A.shape), "incosistent system A no valid"
    x=np.zeros(n)
    r=A@x-b
    k=0
    j=0
    for k in range(maxiter):
        if LA.norm(r,np.inf)<=tol:
            print("Happy iteration ",k)
            break
        alpha=-r[j]/A[j,j]
        x[j] += alpha
        r += alpha*A[:,j]
        j = np.mod(j+1,n)
    #raise Exception('TODO')
    return x


In [ ]:
np.eye(4).shape

In [26]:
def mCGv1( A, b, tol=1e-14 ):
    n=len(b)
    x=np.zeros(n)
    r=-b
    p=-r
    for k in range(n):
        if LA.norm(r,np.inf)<=tol:
            print("Happy iteration",k)
            break
        
        alpha=-r.T@p/(p.T@A@p)
        x += alpha*p
        r += alpha*(A@p)
        beta = r.T@A@p/(p.T@A@p)
        p = -r+ beta*p


    #raise Exception('TODO')
    return x


In [36]:
def mCG( A, b, tol=1e-14 ):
    n=len(b)
    x=np.zeros(n)
    r=-b
    p=-r
    for k in range(n):
        if LA.norm(r,np.inf)<=tol:
            print("Happy iteration",k)
            break

        rsq=r.T@r
        alpha=rsq/(p.T@A@p)
        x += alpha*p
        r += alpha*(A@p)
        beta = r.T@r/rsq
        p = -r+ beta*p
    #raise Exception('TODO')
    return x


## Experimentos

### 1. A diagonal descenso ciclico en n pasos

In [9]:
mDesCyclic(np.diag(np.random.rand(10)), np.ones(10), 1e-10)

Happy iteration  10


array([1.06371524, 1.81653365, 3.65268099, 7.35431483, 1.13014471,
       1.17147004, 1.83972472, 1.11020944, 2.85650396, 1.47617002])

### 2. Comparar versiones de CG (con spline matrix)

In [15]:
# definir A, b
n=50
A=np.diag(2*np.ones(n))+np.diag(np.ones(n-1),-1)+np.diag(np.ones(n-1),1)
b=np.ones(n)
#?np.diag
print(A)

[[2. 1. 0. ... 0. 0. 0.]
 [1. 2. 1. ... 0. 0. 0.]
 [0. 1. 2. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 2. 1. 0.]
 [0. 0. 0. ... 1. 2. 1.]
 [0. 0. 0. ... 0. 1. 2.]]


In [21]:
x=mCGv1(A, b, 1e-6)
print(x, LA.norm(A@x-b, np.inf))

Happy iteration 25
[0.49019608 0.01960784 0.47058824 0.03921569 0.45098039 0.05882353
 0.43137255 0.07843137 0.41176471 0.09803922 0.39215686 0.11764706
 0.37254902 0.1372549  0.35294118 0.15686275 0.33333333 0.17647059
 0.31372549 0.19607843 0.29411765 0.21568627 0.2745098  0.23529412
 0.25490196 0.25490196 0.23529412 0.2745098  0.21568627 0.29411765
 0.19607843 0.31372549 0.17647059 0.33333333 0.15686275 0.35294118
 0.1372549  0.37254902 0.11764706 0.39215686 0.09803922 0.41176471
 0.07843137 0.43137255 0.05882353 0.45098039 0.03921569 0.47058824
 0.01960784 0.49019608] 2.886579864025407e-15


In [38]:
x = mCG(A, b, 1e-6)
print(x, LA.norm(A@x-b, np.inf))

[ 0.99787546  0.01107959 -0.02450102  0.02929108 -0.01991196  0.00728348
 -0.0011183 ] 2.4049536738512955e-05


In [35]:
x = mDesCyclic(A, b, 1e-8)
print(x)

Happy iteration  1
[1. 0. 0. 0. 0. 0. 0.]


### 3. Eigenvalue clusters

In [27]:
def getSymmetricMatrix( d:np.array ):
    """Generates a symmetric matrix A with eigenvalues given by a vector d"""
    assert(len(d.shape)==1),"d must be a one dimensional array"
    n=len(d)
    W=np.random.rand(n,n)
    Q , _ = LA.qr(W)
    A=Q @ np.diag(d) @ Q.T 
    #raise Exception('TODO')
    return A


In [39]:
d = np.ones(10)
A = getSymmetricMatrix( d )
x = mCG( A, np.ones(len(d)) )
x


Happy iteration 1


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [40]:
d = np.hstack( (4*np.ones(5), 6*np.ones(5) ) )
A = getSymmetricMatrix( d )
x = mCG( A, np.ones(len(d)) )
x


Happy iteration 2


array([0.25757309, 0.24138561, 0.24188828, 0.25520232, 0.25554551,
       0.24059449, 0.25469405, 0.26830571, 0.2405315 , 0.22998909])

In [41]:
d = np.hstack( (1,2,3*np.ones(20),4,5))
A = getSymmetricMatrix( d )
x = mCG( A, np.ones(len(d)) )
x


Happy iteration 5


array([1.2275124 , 0.49787468, 0.63293355, 0.44979834, 0.93604833,
       0.56439047, 0.88612772, 0.50466632, 1.0448382 , 0.98311936,
       0.69243366, 1.28998358, 0.76603592, 0.58264015, 1.3247675 ,
       0.96832155, 0.90248354, 0.93893672, 0.50650406, 0.86834042,
       1.24110145, 1.05143002, 0.86079987, 0.83487281])

In [ ]:
d = np.hstack( (1, (5.5+1e-3*(np.random.rand(50)-0.5)), 10 ) )
A = getSymmetricMatrix( d )
x = mCG( A, np.ones(len(d)), tol=1e-6 )


### 4. Pascal
Very bad example, because ill conditioned.

Additionally, the smallest eigenvalue is very close to zero which causes the best polynomial to have steep gradients.

In [42]:
n = 7
A = LA.pascal(n).astype(float)
b = np.ones(n)
x = mCGv1(A, b, 1e-6)
print(x)
print('error = ', LA.norm(A@x-b, np.inf))
x = mCG(A, b, 1e-6)
print(x)
print('error = ', LA.norm(A@x-b, np.inf))
x = mDesCyclic(A, b, 1e-6)
print(x)
print('error = ', LA.norm(A@x-b, np.inf))


[ 0.99787546  0.0110796  -0.02450101  0.02929109 -0.01991195  0.00728348
 -0.00111831]
error =  2.436546928796801e-05
[ 0.99787546  0.01107959 -0.02450102  0.02929108 -0.01991196  0.00728348
 -0.0011183 ]
error =  2.4049536738512955e-05
Happy iteration  1
[1. 0. 0. 0. 0. 0. 0.]
error =  0.0
